In [ ]:
# ML Final Project

In [1]:
# import all th required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
# list of some helper functions
def load_data():
    training_data_path = "fashion-mnist_train.csv"
    testing_data_path = "fashion-mnist_test.csv"

    # preparing train test data
    train_csv = pd.read_csv(training_data_path, header=None, skiprows=1)
    test_csv = pd.read_csv(testing_data_path, header=None, skiprows=1)
    X_train, y_train = train_csv.iloc[:, 1:], train_csv.iloc[:, 0].astype(int)
    X_test, y_test = test_csv.iloc[:, 1:], test_csv.iloc[:, 0].astype(int)  

    # TODO: need to include all the data later
    return (np.asarray(X_train)[:10000],
            np.asarray(y_train)[:10000],
            np.asarray(X_test)[:1000],
            np.asarray(y_test)[:1000])
    
def sigmoid(x):
    # Clip to prevent overflow
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))


In [9]:
class PCA:
    def __init__(self, X, num_components=None): #X = input data, num_components = dimensions to keep
        self.num_components = num_components

        # compute mean of each feature
        self.mn = np.mean(X, 0, keepdims=True)

        # de-mean the features (subtract mean from each sample)
        X_m = X - self.mn

        # compute covariance matrix
        covmat = X_m.T @ X_m/(len(X)-1)

        # compute eigenvalues and eigenvectors
        evals,evects = np.linalg.eigh(covmat)

        # sort by decreasing variance (largest eigenvalue = direction of max var)
        idx = np.argsort(evals)[::-1]
        self.evals = evals[idx]
        self.evects = evects[:,idx]

        # keep only the first n_components if given
        if num_components is not None:
            self.evals = self.evals[:num_components]
            self.evects = self.evects[:, :num_components]

    def transform(self,X):
        # center the data and project onto the principal components
        X_m = X - self.mn
        return X_m @ self.evects

    def num_effective_dims(self,percvar): # gives number of PCs needed for X% variance
        total_var = np.sum(self.evals)
        cumulative_vals = 100 * np.cumsum(self.evals) / total_var
        i = np.searchsorted(cumulative_vals, percvar)
        return i + 1

In [10]:
# The one vs rest strategy for multiclass classification  we studied in class is not very efficient for real time analysis.
# so, we are not using it. Really slow and only give output but not probabilities.

class BinaryLogisticRegression:
    """Binary Class classification"""
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs
        self.mean = None
        self.std = None

    def fit(self, X, y):
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0) + 1e-8  # Add epsilon to avoid division by zero

        X_norm = (X - self.mean) / self.std
        n, d = X_norm.shape

        # Initialize weights
        w = np.zeros(d)

        for _ in range(self.epochs):
            p = sigmoid(X_norm @ w)
            grad = (X_norm.T @ (p - y)) / n
            w -= self.lr * grad

            #  if gradient is close to zero, stop
            if np.linalg.norm(grad) < 1e-6:
                break
        self.w_ = w
        return self

    def predict_probability(self, X):
        X_norm = (X - self.mean) / self.std
        return sigmoid(X_norm @ self.w_)

    def predict(self, X, threshold=0.5):
        return (self.predict_probability(X) >= threshold).astype(int)

class MulticlassLogisticRegression:
    """
    use one vs rest startegy for Multiclass classification. 
    This is slow for real time analysis.
    """
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs
        self.classifiers_ = []

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        y = np.asarray(y)
        for c in self.classes_:
            clf = BinaryLogisticRegression(self.lr, self.epochs)
            # Create binary labels
            y_binary = (y == c).astype(float)
            clf.fit(X, y_binary)
            self.classifiers_.append(clf)
        return self

    def predict(self, X):
        probas = np.stack([clf.predict_probability(X) for clf in self.classifiers_], axis=1)
        return self.classes_[np.argmax(probas, axis=1)]


In [11]:
class LogisticRegression:
    """Better multiclass classification using softmax regression; Similar to multinomial logistic regression"""
    def __init__(self, lr=0.01, epochs=10000, tol=1e-6):
        self.lr = lr
        self.epochs = epochs
        self.tol = tol

    def softmax(self, x):
        z = x - np.max(x, axis=1, keepdims=True)  
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y)

        n_samples, n_features = X.shape
        classes, y_idx = np.unique(y, return_inverse=True)
        n_classes = len(classes)
        self.classes_ = classes

        # Normalization
        self.mean = X.mean(axis=0)
        std = X.std(axis=0)
        self.std = np.where(std == 0, 1e-8, std)
        X = (X - self.mean) / self.std

        # y matrix
        Y = np.eye(n_classes, dtype=np.float64)[y_idx]

        # Weights
        self.W = np.zeros((n_features, n_classes), dtype=np.float64)

        # Gradient Descent
        for _ in range(self.epochs):
            probs = self.softmax((X @ self.W))
            grad = (X.T @ (probs - Y)) / n_samples

            # updating the weights
            self.W -= self.lr * grad

            if np.linalg.norm(grad) < self.tol:
                break

        return self

    def _standardize(self, X):
        return (X - self.mean) / self.std

    def predict_proba(self, X):
        X = np.asarray(X, dtype=np.float64)
        Xn = self._standardize(X)
        return self.softmax((Xn @ self.W))

    def predict(self, X):
        probs = self.predict_proba(X)
        idx = np.argmax(probs, axis=1)
        return self.classes_[idx]

In [12]:
# Load data
X_train, y_train, X_test, y_test = load_data()

# PCA
pca = PCA(X_train, num_components=100) # pick # components
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Train model (use better model than one vs many)
model = LogisticRegression(lr=0.01, epochs=50000)
model.fit(X_train_pca, y_train) # change X_train_pca to X_train for no PCA

# Predict
y_pred = model.predict(X_test_pca) # change X_test_pca to X_test for no PCA

# save this model to be used later by app.
with open("fashion-mnist-model.pkl", 'wb') as f:
  pickle.dump(model, f)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8350


In [7]:
# testing with RBF kernels just to see if we get better accuracy.( The accuracy is even bad)
class KMeans:
    def __init__(self, n_clusters=50, max_iter=100, tol=1e-4, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

    def fit(self, X):
        rng = np.random.default_rng(self.random_state)
        n_samples, n_features = X.shape

        # Initialize centers randomly
        idx = rng.choice(n_samples, self.n_clusters, replace=False)
        centers = X[idx].copy()

        for _ in range(self.max_iter):
            # Assign points to closest center
            dists = np.linalg.norm(X[:, None, :] - centers[None, :, :], axis=2)
            labels = np.argmin(dists, axis=1)

            # Compute new centers
            new_centers = np.array([X[labels == i].mean(axis=0) if np.any(labels == i) else centers[i] 
                                    for i in range(self.n_clusters)])
            
            # Check convergence
            if np.linalg.norm(new_centers - centers) < self.tol:
                break
            centers = new_centers

        self.cluster_centers_ = centers
        return self

class RBFTransformer:
    def __init__(self, n_centers=50, sigma=None, random_state=None):
        self.n_centers = n_centers
        self.sigma = sigma
        self.random_state = random_state

    def fit(self, X):
        # Compute centers with simple KMeans
        kmeans = KMeans(n_clusters=self.n_centers, random_state=self.random_state)
        kmeans.fit(X)
        self.centers = kmeans.cluster_centers_

        # sigma = median distance to nearest center
        dists = np.linalg.norm(X[:, None, :] - self.centers[None, :, :], axis=2)
        min_dists = np.min(dists, axis=1)
        if self.sigma is None:
            self.sigma = np.median(min_dists)

        return self

    def transform(self, X):
        dists = np.linalg.norm(X[:, None, :] - self.centers[None, :, :], axis=2)
        return np.exp(-dists**2 / (2 * self.sigma**2))

    def fit_transform(self, X):
        return self.fit(X).transform(X)


In [13]:
rbf = RBFTransformer(n_centers=10)
X_train_rbf = rbf.fit_transform(X_train)
X_test_rbf = rbf.transform(X_test)

model_rbf = LogisticRegression(lr=0.01, epochs=1000)
model_rbf.fit(X_train_rbf, y_train)
y_pred_rbf = model_rbf.predict(X_test_rbf)  

accuracy_rbf = np.mean(y_pred_rbf == y_test)
print(f"RBF Kernel Accuracy: {accuracy_rbf:.4f}")

RBF Kernel Accuracy: 0.5710
